tgb - 11/13/2019 - Continuity of 028 but for simultaneous training while files are pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog
/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# 1) NN with only q and T as inputs

## 1.1) Rescaling

In [2]:
scale_dict = {
    'PHQ': L_V/G,  
    'TPHYSTND': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [3]:
# Takes representative value for PS since purpose is normalization
PS = 1e5; P0 = 1e5;
P = P0*hyai+PS*hybi; # Total pressure [Pa]
dP = P[1:]-P[:-1]; # Differential pressure [Pa]

In [4]:
for v in ['PHQ','TPHYSTND']:
    scale_dict[v] *= dP

In [5]:
save_pickle('./nn_config/scale_dicts/100_POG_scaling.pkl', scale_dict)

In [6]:
from cbrain.data_generator import DataGenerator

In [20]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

OSError: [Errno -101] NetCDF: HDF error: b'/local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc'

In [22]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/101_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [24]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 1.2) Model and training

In [14]:
inp = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

In [15]:
NNmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 128)               0   

In [27]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG101.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [28]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

In [ ]:
Nep = 10
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/10
41376/41376 [==============================] - 436s 11ms/step - loss: 540.1531 - val_loss: 543.7227
Epoch 2/10
41376/41376 [==============================] - 406s 10ms/step - loss: 526.0911 - val_loss: 525.1363
Epoch 3/10
41376/41376 [==============================] - 239s 6ms/step - loss: 517.9633 - val_loss: 523.7054
Epoch 4/10
41376/41376 [==============================] - 261s 6ms/step - loss: 513.0710 - val_loss: 520.2624
Epoch 5/10
41376/41376 [==============================] - 353s 9ms/step - loss: 509.8734 - val_loss: 516.3279
Epoch 6/10
41376/41376 [==============================] - 265s 6ms/step - loss: 507.3517 - val_loss: 512.4247
Epoch 7/10
41376/41376 [==============================] - 251s 6ms/step - loss: 505.3103 - val_loss: 516.4698
Epoch 8/10
41376/41376 [==============================] - 414s 10ms/step - loss: 503.3925 - val_loss: 508.9910
Epoch 9/10
41376/41376 [==============================] - 430s 10ms/step - loss: 501.6747 - val_loss: 515.9987
Epoch 